In [3]:
import pandas as pd
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

In [ ]:
DATASET_PATH = os.getenv("DATASET_PATH")

In [ ]:
df = pd.read_csv('DATASET_PATH')

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Checking and droping null values

In [10]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [11]:
df = df.dropna()

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


### Splitting independent and dependent features

In [13]:
X = df.drop('label', axis = 1)
Y = df['label']

In [14]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
Y.head()

,label
0,1
1,0
2,1
3,1
4,1


In [16]:
import tensorflow as tf
tf.__version__

'2.18.0'

In [32]:
pip install --upgrade tensorflow

In [33]:
tf.__version__

'2.18.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
vocab_size = 5000

### Onehot representation

In [19]:
messages = X.copy()

In [20]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


### Text Preprocessing

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
#Resetting the index
messages = messages.reset_index(drop=True)

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
one_hot_repr = [one_hot(words,vocab_size) for words in corpus]
one_hot_repr

[[4148, 2866, 191, 2754, 3796, 3814, 2659, 3381, 983, 3625],
 [4985, 4376, 2627, 2166, 2019, 3681, 4440],
 [342, 188, 2699, 3407],
 [3185, 3863, 4777, 4160, 3070, 2438],
 [2080, 2019, 2877, 4500, 2020, 3496, 2019, 3496, 12, 140],
 [4498,
  1780,
  3550,
  2352,
  4616,
  2939,
  322,
  3852,
  1908,
  4861,
  4282,
  1546,
  204,
  766,
  4440],
 [3806, 1025, 529, 3184, 2596, 35, 406, 1108, 1965, 3599, 4773],
 [1453, 787, 2486, 353, 1594, 3036, 2939, 4613, 1965, 3599, 4773],
 [3157, 423, 3053, 2156, 1163, 1965, 601, 86, 2939, 938],
 [582, 2456, 3671, 3568, 3128, 3666, 4964, 123],
 [1046, 4806, 2029, 120, 1137, 3387, 939, 3821, 2641, 537, 4726],
 [4160, 3544, 3796, 1965, 2939, 1594],
 [4495, 3320, 2298, 2942, 552, 3665, 878, 724, 3394],
 [4467, 2210, 2394, 4778, 3629, 3971, 341, 1965, 3599, 4773],
 [2664, 4829, 1862, 4175, 3253, 1965, 3599, 4773],
 [1663, 2761, 3152, 1676, 2701, 1600, 2968, 1986, 4874, 2363],
 [37, 3837, 4376],
 [2616, 1850, 997, 3911, 2939, 1174, 976, 4440],
 [2774, 49

### Now we need to add padding, so that are model can be trained


In [28]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_repr,padding = 'post',maxlen = sent_length)
print(embedded_docs)

[[4148 2866  191 ...    0    0    0]
 [4985 4376 2627 ...    0    0    0]
 [ 342  188 2699 ...    0    0    0]
 ...
 [ 600 4432  149 ...    0    0    0]
 [4345 1594 2211 ...    0    0    0]
 [2211 4626 3606 ...    0    0    0]]


In [29]:
embedded_docs[1]

array([4985, 4376, 2627, 2166, 2019, 3681, 4440,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

### Creating LSTM model


In [35]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Force the model to build by passing a sample input
dummy_input = tf.random.uniform((1, sent_length), minval=0, maxval=vocab_size, dtype=tf.int32)
model(dummy_input)


# Print the summary again
print(model.summary())


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (1, 20, 40)                 │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (1, 100)                    │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (1, 1)                      │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


# Force the model to build by passing a sample input
dummy_input = tf.random.uniform((1, sent_length), minval=0, maxval=vocab_size, dtype=tf.int32)
model(dummy_input)

This happend because TensorFlow/Keras lazy-builds models—meaning the actual layer shapes and parameters aren't initialized until the model receives input data for the first time. Hence above we passing an input so that the model gets initialsed

In [36]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)


In [37]:
X_final.shape,Y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test , Y_train, Y_test = train_test_split(X_final,Y_final,test_size = 0.33, random_state = 42)

###Training the model

In [41]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs = 10, batch_size = 64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.7589 - loss: 0.4262 - val_accuracy: 0.9234 - val_loss: 0.1869
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9530 - loss: 0.1240 - val_accuracy: 0.9180 - val_loss: 0.2075
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.9727 - loss: 0.0788 - val_accuracy: 0.9115 - val_loss: 0.2600
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9805 - loss: 0.0607 - val_accuracy: 0.9148 - val_loss: 0.2723
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.9887 - loss: 0.0360 - val_accuracy: 0.9021 - val_loss: 0.3193
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.9886 - loss: 0.0377 - val_accuracy: 0.9069 - val_loss: 0.3117
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.9905 - loss: 0.0282 - val_accuracy: 0.9070 - val_loss: 0.3169
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9936 - loss: 0.0235 - val

###Performance metrics and accuracy

In [42]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


Convering probabilities to class labels


In [44]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [49]:
from sklearn.metrics import confusion_matrix

Y_test → The actual class labels (ground truth) from the test dataset.

y_pred → The predicted class labels output by the model.

In [51]:
confusion_matrix(Y_test,y_pred)

array([[3158,  261],
       [ 356, 2260]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

0.8977630488815245

In [55]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3419
           1       0.90      0.86      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.89      0.90      6035
weighted avg       0.90      0.90      0.90      6035



In [57]:
model.save("fake_news_classifier.keras")

In [60]:
from tensorflow.keras.models import load_model
model = load_model("fake_news_classifier.keras")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
